In [29]:
import matplotlib as plt
import pandas as pd
import requests
import seaborn
import glob
import statsmodels.api as sm
from datetime import date
import numpy as np

In [30]:
sectors = ['banks']#'fmcg','pharma','housing']

In [76]:
def cointegration_test_ct(S1, S2,date1,date2):
    i = S1.index.intersection(S2.index)
    length = len(i) -1
    print(i[0])
    print(i[length])
    S11=S1[i[0]:i[length]]
    S22=S2[i[0]:i[length]]
    x = (S11[date1:date2]['Adjclose'])
    y = (S22[date1:date2]['Adjclose'])
    if len(x) == 0 or len(y) == []:
        return [[0,0,0]]
    # Step 1: regress on variable on the other
    ols_result = sm.OLS(y, x).fit()
    return sm.tsa.stattools.adfuller(ols_result.resid,regression  = 'ct'),ols_result.resid

In [77]:
def cointegration_test(S1, S2,date1,date2):
    i = S1.index.intersection(S2.index)
    length = len(i) -1
    S11=S1[i[0]:i[length]]
    S22=S2[i[0]:i[length]]
    x = (S11[date1:date2]['Adjclose'])
    y = (S22[date1:date2]['Adjclose'])
    if len(x) == 0 or len(y) == []:
        return [[0,0,0]]
    # Step 1: regress on variable on the other
    ols_result = sm.OLS(y, x).fit()
    return sm.tsa.stattools.adfuller(ols_result.resid),ols_result.resid

In [78]:
pairs = []
df = pd.read_excel('pairs.xlsx')
for i in range(204):
    first = '2013/' + str(df['first'][i]) + '.csv'
    second = '2013/' + str(df['second'][i]) + '.csv'
    pairs.append((first,second))
df = pd.read_csv(pairs[0][1],sep = ';')
print(df)

'''for name in sectors:
    list_of_companies = glob.glob(name+'/*.csv')
    n = len(list_of_companies)
    for i in range(n):
        print(i)
        for j in range(i+1, n):
            print(j)
            pairs.append((list_of_companies[i],list_of_companies[j]))'''

         Symbol  Adjclose     Close      High       Low      Open   Timestamp  \
0     GRASIM.BO   16.6871   60.8390   61.6578   59.8198   59.8198  2000-02-16   
1     GRASIM.BO   16.4992   60.1539   61.3236   59.1514   60.8223  2000-02-17   
2     GRASIM.BO   15.5116   56.5531   60.4881   55.8930   59.3268  2000-02-18   
3     GRASIM.BO   14.2719   52.0332   58.4830   52.0332   57.6475  2000-02-21   
4     GRASIM.BO   13.4721   49.1174   52.4843   48.6244   52.4843  2000-02-22   
...         ...       ...       ...       ...       ...       ...         ...   
5004  GRASIM.BO  758.9500  758.9500  790.0000  752.1000  790.0000  2020-02-10   
5005  GRASIM.BO  758.1000  758.1000  768.5000  754.5000  759.0000  2020-02-11   
5006  GRASIM.BO  755.1000  755.1000  774.1500  753.9500  760.8000  2020-02-12   
5007  GRASIM.BO  752.2000  752.2000  764.6500  746.0000  756.9500  2020-02-13   
5008  GRASIM.BO  743.5000  743.5000  770.8000  737.3500  752.2000  2020-02-14   

         Volume  
0      95

"for name in sectors:\n    list_of_companies = glob.glob(name+'/*.csv')\n    n = len(list_of_companies)\n    for i in range(n):\n        print(i)\n        for j in range(i+1, n):\n            print(j)\n            pairs.append((list_of_companies[i],list_of_companies[j]))"

In [57]:
def trade(S1, S2, window1, window2):
    
    # If window length is 0, algorithm doesn't make sense, so exit
    if (window1 == 0) or (window2 == 0):
        return 0
    
    # Compute rolling mean and rolling standard deviation
    ratios = S1/S2
    ma1 = ratios.rolling(window=window1,
                               center=False).mean()
    ma2 = ratios.rolling(window=window2,
                               center=False).mean()
    std = ratios.rolling(window=window2,
                        center=False).std()
    zscore = (ma1 - ma2)/std
    
    #zscore = zscore[]
    # Simulate trading
    # Start with no money and no positions
    money = 10000
    returns = []
    countS1 = 0
    countS2 = 0
    for i in zscore.index:
        # Sell short if the z-score is > 1
        if zscore[i] > 1 and (countS1==0) and (countS2==0) :
            countS1 = -(money/2)/S1[i]
            countS2 = (money/2)/S2[i]
            #print(S1[i])
            #print(S2[i])
            #print('Selling Ratio %s %s %s %s'%(money, ratios[i], countS1,countS2))
            #print('Selling Ratio %s %s %s %s %s'%(money, ratios[i], countS1,countS2,zscore[i]))
        # Buy long if the z-score is < 1
        elif zscore[i] < -1 and (countS1==0) and (countS2==0):
            countS1 = (money/2)/S1[i]
            countS2 = -(money/2)/S2[i]
            #print(S1[i])
            #print(S2[i])
            #print('Buying Ratio %s %s %s %s'%(money,ratios[i], countS1,countS2))
            #print('Buying Ratio %s %s %s %s %s'%(money,ratios[i], countS1,countS2,zscore[i]))
        # Clear positions if the z-score between -.5 and .5
        #elif abs(zscore[i]) < 1:
        elif abs(zscore[i]) < 0.5:
            money += S1[i] * countS1 + S2[i] * countS2
            countS1 = 0
            countS2 = 0
            #print('Exit pos %s %s %s %s'%(money,ratios[i], countS1,countS2))
            #print(S1[i])
            #print(S2[i])
            #returns.append(money)
            
            
            #print('Exit pos %s %s %s %s %s'%(money,ratios[i], countS1,countS2,zscore[i]))
            
            
    return money

In [58]:
def trade1(data1, data2,date1,date2,window):
    S1 = data1[date1:date2]['Adjclose']
    S2 = data2[date1:date2]['Adjclose']
    window1 = window[0] + 3
    window2 = window[1] + 35
    # If window length is 0, algorithm doesn't make sense, so exit
    if (window1 == 0) or (window2 == 0):
        return 0

    # Compute rolling mean and rolling standard deviation
    ratios = S1/S2
    ma1 = ratios.rolling(window=window1,
                               center=False).mean()
    ma2 = ratios.rolling(window=window2,
                               center=False).mean()
    std = ratios.rolling(window=window2,
                        center=False).std()
    zscore = (ma1 - ma2)/std
    
   
    money = 10000
    returns = [10000]
    countS1 = 0
    countS2 = 0
    for i in zscore.index:
        # Sell short if the z-score is > 1
        if zscore[i] > 1 and (countS1==0) and (countS2==0) :
            countS1 = -(money/2)/S1[i]
            countS2 = (money/2)/S2[i]
            #print(S1[i])
            #print(S2[i])
            print(i)
            print('Selling Ratio %s %s %s %s'%(money, ratios[i], countS1,countS2))
            #print('Selling Ratio %s %s %s %s %s'%(money, ratios[i], countS1,countS2,zscore[i]))
        # Buy long if the z-score is < 1
        elif zscore[i] < -1 and (countS1==0) and (countS2==0):
            countS1 = (money/2)/S1[i]
            countS2 = -(money/2)/S2[i]
            #print(S1[i])
            #print(S2[i])
            print(i)
            print('Buying Ratio %s %s %s %s'%(money,ratios[i], countS1,countS2))
        # Clear positions if the z-score between -.5 and .5
        #elif abs(zscore[i]) < 1:
        elif abs(zscore[i]) < 0.5:
            money += S1[i] * countS1 + S2[i] * countS2
            countS1 = 0
            countS2 = 0
            print(i)
            #print(S1[i])
            #print(S2[i])
            print('Exit pos %s %s %s %s'%(money,ratios[i], countS1,countS2))
            returns.append(money)
            
    print('test' + str(returns[-1]))
    #testing.append(returns[-1])
    #return pd.Series(returns)
    return returns[-1]

In [59]:
def optimal_window(name1,name2):
    main = []
    data1 = (pd.read_csv(name1,parse_dates = ['Date'],index_col = 'Date')).interpolate(axis=0)
    data2 = (pd.read_csv(name2,parse_dates = ['Date'],index_col = 'Date')).interpolate(axis=0)
    start_date = '20080701'
    end_date = '20130630'
    set1 = (data1[start_date:end_date])
    set2 = (data2[start_date:end_date])    
    for f in range(3,25):
        row =  [trade(set1['Adj Close'],set2['Adj Close'],f,s) for s in range(35,200)] 
        main.append(row)
    scores = np.array(main)
    windows = np.unravel_index(np.argmax(scores),shape=(22,165))
    print(np.argmax(scores))
    print(windows)
    print('training' + str(scores[windows[0],windows[1]]))
    training.append(scores[windows[0],windows[1]])
    #plt.figure.Figure(figsize=(25,25))
    #seaborn.heatmap(scores)
    cumsum = 0
    count = 1
    for i in np.nditer(scores):
        cumsum = cumsum + i
        count = count + 1
    print('Avg Return')
    print(cumsum/count)
    print(np.mean(scores))
    avg.append(np.mean(scores))
    return windows

In [60]:
def avg_optimal(S1,S2,date1,date2):
    main = []
    set1 = (S1[date1:date2])
    set2 = (S2[date1:date2])
    for f in range(3,25):
        row =  [trade(set1['Adjclose'],set2['Adjclose'],f,s) for s in range(35,200)] 
        main.append(row)
    scores = np.array(main)
    row_avg = np.mean(scores,axis=1)
    col_avg = np.mean(scores,axis=0)
    fma = np.argmax(row_avg) + 3
    sma = np.argmax(col_avg) + 35
    print(fma)
    print(sma)
    return (fma,sma)

In [61]:
periods = [("20080701","20130630"),("20100101","20150101"),("20110101","20160101"),("20120101","20170101")
           ,("20130101","20180101")]
test = [('20130630','20140630'),('20150101','20160101'),('20160101','20170101'),('20170101','20180101'),
        ('20180101','20190101'),]

In [67]:
avg = []
training = []
testing = []

for item in pairs[149:152]:
    print(item)
    name1 = item[0]
    name2 = item[1]
    data1 = (pd.read_csv(name1,sep=';',parse_dates = ['Timestamp'],index_col = 'Timestamp')).interpolate(axis=0)
    data2 = (pd.read_csv(name2,sep=';',parse_dates = ['Timestamp'],index_col = 'Timestamp')).interpolate(axis=0)
    money = 0
    returns = []
    train_p = periods[0]
    test_p = test[0]
    print(train_p)
    window = avg_optimal(data1,data2,train_p[0],train_p[1])
    print(test_p)
    temp = trade1(data1,data2,test_p[0],test_p[1],window)
    print("Return for" + test_p[0]+test_p[1] + "is" + str(temp))
    returns.append(temp)
    money += temp
training.append(returns)

('2013/HCLTECH.BO.csv', '2013/OFSS.BO.csv')
('20080701', '20130630')


KeyboardInterrupt: 

In [80]:
avg = []
arr = []
arr_ct = []

for item in pairs:
    print(item)
    name1 = item[0]
    name2 = item[1]
    data1 = (pd.read_csv(name1,sep=';',parse_dates = ['Timestamp'],index_col = 'Timestamp')).interpolate(axis=0)
    data2 = (pd.read_csv(name2,sep=';',parse_dates = ['Timestamp'],index_col = 'Timestamp')).interpolate(axis=0)
    
    money = 0
    returns = []
    train_p = periods[0]
    test_p = test[0]
    value1 = cointegration_test(data1,data2,train_p[0],train_p[1])[0][1]
    value2 = cointegration_test_ct(data1,data2,train_p[0],train_p[1])[0][1]
    print(train_p)
    print(str(value1) +  "  " + str(value2))
    print("---")
    arr.append(value1)
    arr_ct.append(value2)

('2013/CENTURYTEX.BO.csv', '2013/GRASIM.BO.csv')
2000-02-16 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.49042048563779  0.470140351653776
---
('2013/ACC.BO.csv', '2013/AMBUJACEM.BO.csv')
2000-02-16 00:00:00
2020-02-13 00:00:00
('20080701', '20130630')
0.18505707475858024  0.08594541026489577
---
('2013/ACC.BO.csv', '2013/ULTRACEMCO.BO.csv')
2000-02-16 00:00:00
2020-02-13 00:00:00
('20080701', '20130630')
0.9334956363631858  0.7941853788849436
---
('2013/AMBUJACEM.BO.csv', '2013/ULTRACEMCO.BO.csv')
2000-02-16 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.47616859700689507  0.477398361607805
---
('2013/JSWENERGY.BO.csv', '2013/NHPC.BO.csv')
2010-01-04 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.4797217046419089  0.14916937160755467
---
('2013/JSWENERGY.BO.csv', '2013/NTPC.BO.csv')
2010-01-04 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.7591402223954226  0.7686980615578746
---
('2013/JSWENERGY.BO.csv', '2013/POWERGRID.BO.csv')
2010-01-04 00:00:

2002-01-14 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.8490177887267778  0.04227791112110817
---
('2013/HDFC.BO.csv', '2013/KOTAKBANK.BO.csv')
2000-02-16 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.035622099524269216  0.048603659136498716
---
('2013/HDFC.BO.csv', '2013/RECLTD.BO.csv')
2008-03-12 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.539828008621899  0.681273739835637
---
('2013/HDFC.BO.csv', '2013/SRTRANSFIN.BO.csv')
2000-03-06 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.27617360447073036  0.5874741984512836
---
('2013/HDFC.BO.csv', '2013/YESBANK.BO.csv')
2005-07-12 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.28417347976904056  0.051881780611959305
---
('2013/HDFCBANK.BO.csv', '2013/ICICIBANK.BO.csv')
2000-02-16 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.36905882101082266  0.5304816431544591
---
('2013/HDFCBANK.BO.csv', '2013/IDFC.BO.csv')
2005-08-12 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.86

2002-09-24 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.7456247917301843  0.3753406280447467
---
('2013/BHEL.BO.csv', '2013/HAVELLS.BO.csv')
2002-07-01 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.9981254406545391  0.6878874690645193
---
('2013/BHEL.BO.csv', '2013/SIEMENS.BO.csv')
2000-02-16 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.6358105287758602  0.19131920432297506
---
('2013/HAVELLS.BO.csv', '2013/SIEMENS.BO.csv')
2002-07-01 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.9929854424708255  0.9423738980713834
---
('2013/BHARTIARTL.BO.csv', '2013/IDEA.BO.csv')
2007-03-09 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.997826258415215  0.850636469287086
---
('2013/BHARTIARTL.BO.csv', '2013/RCOM.BO.csv')
2002-02-20 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
8.118178336497622e-05  0.0019240854976445563
---
('2013/IDEA.BO.csv', '2013/RCOM.BO.csv')
2007-03-09 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.0192520771

2010-11-04 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.012038085041833254  0.07352516931215461
---
('2013/COALINDIA.BO.csv', '2013/NMDC.BO.csv')
2010-11-04 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.5596954238266622  0.03810296901160257
---
('2013/COALINDIA.BO.csv', '2013/SAIL.BO.csv')
2010-11-04 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.16793718760486676  0.4378079802251793
---
('2013/COALINDIA.BO.csv', '2013/VEDL.BO.csv')
2010-11-04 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.31893117858240705  0.27291460137048024
---
('2013/COALINDIA.BO.csv', '2013/STRTECH.BO.csv')
2010-11-04 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.08163102106539083  0.10872246760670473
---
('2013/COALINDIA.BO.csv', '2013/TATASTEEL.BO.csv')
2010-11-04 00:00:00
2020-02-14 00:00:00
('20080701', '20130630')
0.6298575200044223  0.2665859702203379
---
('2013/HINDALCO.BO.csv', '2013/HINDZINC.BO.csv')
2000-02-16 00:00:00
2020-02-14 00:00:00
('20080701', '20

In [82]:
df = pd.DataFrame(list(zip(pairs,arr,arr_ct)), 
               columns =['Pairs','coint', 'coint_ct']) 
df.to_csv('ct_values.csv')

In [31]:
avg = []
training = []
testing = []

for item in pairs:
    print(item)
    name1 = item[0]
    name2 = item[1]
    data1 = (pd.read_csv(name1,parse_dates = ['Date'],index_col = 'Date')).interpolate(axis=0)
    data2 = (pd.read_csv(name2,parse_dates = ['Date'],index_col = 'Date')).interpolate(axis=0)
    money = 0
    returns = []
    for i in range(0,5):
        train_p = periods[i]
        test_p = test[i]
        print(train_p)
        window = avg_optimal(data1,data2,train_p[0],train_p[1])
        print(test_p)
        temp = trade1(data1,data2,test_p[0],test_p[1],window)
        print("Return for" + test_p[0]+test_p[1] + "is" + str(temp))
        returns.append(temp)
        money += temp
    training.append(returns)
    avg.append(money/5)
    print("Avg amount over 5 years " + str(money/5))
        
        

('banks\\ALBK.BO.csv', 'banks\\ANDHRABANK.BO.csv')
('20090101', '20140101')
3
179
('20140101', '20150101')
2014-11-26 00:00:00
Exit pos 10000.0 1.5071412428133546 0 0
2014-11-27 00:00:00
Exit pos 10000.0 1.4775157798417826 0 0
2014-11-28 00:00:00
Exit pos 10000.0 1.4860828169844567 0 0
2014-12-01 00:00:00
Exit pos 10000.0 1.490413665997854 0 0
2014-12-02 00:00:00
Exit pos 10000.0 1.5246096292775497 0 0
2014-12-03 00:00:00
Exit pos 10000.0 1.5823224425508646 0 0
2014-12-04 00:00:00
Exit pos 10000.0 1.6068797151981642 0 0
2014-12-05 00:00:00
Exit pos 10000.0 1.5826695955090022 0 0
2014-12-08 00:00:00
Exit pos 10000.0 1.5638196924571912 0 0
2014-12-12 00:00:00
Exit pos 10000.0 1.5139462127902683 0 0
2014-12-15 00:00:00
Exit pos 10000.0 1.51067819798061 0 0
2014-12-16 00:00:00
Exit pos 10000.0 1.5242060174196201 0 0
2014-12-17 00:00:00
Exit pos 10000.0 1.49473151422017 0 0
2014-12-18 00:00:00
Exit pos 10000.0 1.5129400531903503 0 0
2014-12-19 00:00:00
Exit pos 10000.0 1.5419835265315491 0 

3
42
('20160101', '20170101')
2016-04-27 00:00:00
Exit pos 10000.0 0.1305542618028706 0 0
2016-04-28 00:00:00
Exit pos 10000.0 0.12847983485814737 0 0
2016-04-29 00:00:00
Exit pos 10000.0 0.12598694838293892 0 0
2016-05-02 00:00:00
Exit pos 10000.0 0.12294459845223321 0 0
2016-05-03 00:00:00
Exit pos 10000.0 0.1225296787455949 0 0
2016-05-04 00:00:00
Exit pos 10000.0 0.11821281026795914 0 0
2016-05-05 00:00:00
Exit pos 10000.0 0.11844338860228562 0 0
2016-05-11 00:00:00
Buying Ratio 10000.0 0.10944568969884513 93.54536775406234 -10.23813729197546
2016-06-22 00:00:00
Exit pos 10474.076493621986 0.11946983734435367 0 0
2016-06-23 00:00:00
Exit pos 10474.076493621986 0.12356934250078681 0 0
2016-06-24 00:00:00
Exit pos 10474.076493621986 0.13017926625364176 0 0
2016-06-30 00:00:00
Selling Ratio 10474.076493621986 0.13219445909567856 -75.78926770462414 10.018921249470369
2016-08-17 00:00:00
Exit pos 10607.08273279653 0.1291836930795955 0 0
2016-08-18 00:00:00
Exit pos 10607.08273279653 0.1

5
55
('20140101', '20150101')
2014-05-13 00:00:00
Buying Ratio 10000 0.5930913234068055 52.56258640303657 -31.17441393146152
2014-05-26 00:00:00
Exit pos 11583.400143753614 0.7741270427116862 0 0
2014-05-27 00:00:00
Exit pos 11583.400143753614 0.7996070698473141 0 0
2014-05-28 00:00:00
Exit pos 11583.400143753614 0.7896168211825405 0 0
2014-05-30 00:00:00
Selling Ratio 11583.400143753614 0.8053944602679296 -46.66993248381175 37.58770508354028
2014-07-16 00:00:00
Exit pos 12069.798674862352 0.7413117548325334 0 0
2014-07-17 00:00:00
Exit pos 12069.798674862352 0.7370474288819753 0 0
2014-07-18 00:00:00
Exit pos 12069.798674862352 0.7351616386345506 0 0
2014-07-21 00:00:00
Exit pos 12069.798674862352 0.7365913581614456 0 0
2014-07-22 00:00:00
Exit pos 12069.798674862352 0.7341452697158757 0 0
2014-07-23 00:00:00
Exit pos 12069.798674862352 0.7145533301165901 0 0
2014-07-24 00:00:00
Exit pos 12069.798674862352 0.6722300358590096 0 0
2014-07-25 00:00:00
Exit pos 12069.798674862352 0.697385

3
199
('20150101', '20160101')
2015-12-11 00:00:00
Selling Ratio 10000 0.5875216795923452 -73.74630942125474 43.32755557491238
test10000
Return for2015010120160101is10000
('20110101', '20160101')
3
160
('20160101', '20170101')
2016-11-09 00:00:00
Exit pos 10000.0 0.6232938922057526 0 0
2016-11-10 00:00:00
Exit pos 10000.0 0.5972613838540253 0 0
2016-11-11 00:00:00
Exit pos 10000.0 0.6063088748961358 0 0
2016-11-15 00:00:00
Exit pos 10000.0 0.5910750344827587 0 0
2016-11-16 00:00:00
Exit pos 10000.0 0.5858085953582985 0 0
2016-11-17 00:00:00
Exit pos 10000.0 0.5903212447279307 0 0
2016-12-22 00:00:00
Buying Ratio 10000.0 0.5687732540664173 81.69934507027214 -46.468402350713795
test10000.0
Return for2016010120170101is10000.0
('20120101', '20170101')
13
199
('20170101', '20180101')
2017-12-08 00:00:00
Buying Ratio 10000 0.4023764645652649 67.11409395973155 -27.005131850017776
test10000
Return for2017010120180101is10000
('20130101', '20180101')
3
57
('20180101', '20190101')
2018-05-16 00:0

14
198
('20160101', '20170101')
2016-12-13 00:00:00
Buying Ratio 10000 0.23344712863289865 72.41129301616395 -16.90420843521894
test10000
Return for2016010120170101is10000
('20120101', '20170101')
14
104
('20170101', '20180101')
2017-07-24 00:00:00
Buying Ratio 10000 0.2142149321401376 66.0938515242868 -14.158289919155429
2017-09-25 00:00:00
Exit pos 9858.948327945345 0.20765448967677186 0 0
2017-09-26 00:00:00
Exit pos 9858.948327945345 0.20656489160305344 0 0
2017-09-27 00:00:00
Exit pos 9858.948327945345 0.20941662731927638 0 0
2017-09-28 00:00:00
Exit pos 9858.948327945345 0.20920303443884752 0 0
2017-09-29 00:00:00
Exit pos 9858.948327945345 0.213764001185418 0 0
2017-10-03 00:00:00
Exit pos 9858.948327945345 0.21371817972333687 0 0
2017-10-04 00:00:00
Exit pos 9858.948327945345 0.21453003885205907 0 0
2017-10-05 00:00:00
Exit pos 9858.948327945345 0.21326018854937162 0 0
2017-10-06 00:00:00
Exit pos 9858.948327945345 0.21379857890563045 0 0
2017-10-09 00:00:00
Exit pos 9858.94832

...

3
192
('20160101', '20170101')
2016-12-05 00:00:00
Buying Ratio 10000 2.856665722282446 6.880118310074133 -19.654198141636602
test10000
Return for2016010120170101is10000
('20120101', '20170101')
3
85
('20170101', '20180101')
2017-06-27 00:00:00
Selling Ratio 10000 3.481279147772638 -5.140492893878751 17.895490720733516
2017-08-04 00:00:00
Exit pos 10340.574003364358 3.2643756353821978 0 0
2017-08-07 00:00:00
Exit pos 10340.574003364358 3.204929311380341 0 0
2017-08-08 00:00:00
Exit pos 10340.574003364358 3.2478667089445437 0 0
2017-08-09 00:00:00
Exit pos 10340.574003364358 3.2816317922581684 0 0
2017-08-10 00:00:00
Exit pos 10340.574003364358 3.3475237571076892 0 0
2017-08-16 00:00:00
Selling Ratio 10340.574003364358 3.536585440860773 -5.167703150107125 18.276023723359213
2017-10-30 00:00:00
Exit pos 10734.594476951577 3.29228319848222 0 0
2017-10-31 00:00:00
Exit pos 10734.594476951577 3.346880088824595 0 0
2017-11-06 00:00:00
Buying Ratio 10734.594476951577 3.0632086342650746 5.3265

3
79
('20150101', '20160101')
2015-06-17 00:00:00
Buying Ratio 10000 4.520009682474025 7.751924043051335 -35.038771732395226
2015-07-08 00:00:00
Exit pos 10583.09051538978 5.059255514848933 0 0
2015-07-09 00:00:00
Exit pos 10583.09051538978 5.02794786214548 0 0
2015-07-10 00:00:00
Exit pos 10583.09051538978 5.001729348870023 0 0
2015-07-15 00:00:00
Selling Ratio 10583.09051538978 5.0515076639773815 -7.442945319009883 37.598095321543
2015-08-04 00:00:00
Exit pos 10969.441754639496 4.698143344437506 0 0
2015-08-05 00:00:00
Exit pos 10969.441754639496 4.754994392421137 0 0
2015-08-06 00:00:00
Exit pos 10969.441754639496 4.86778598410385 0 0
2015-08-07 00:00:00
Exit pos 10969.441754639496 4.888807204347102 0 0
2015-08-10 00:00:00
Exit pos 10969.441754639496 4.938225038216918 0 0
2015-08-11 00:00:00
Exit pos 10969.441754639496 5.024387607597446 0 0
2015-08-12 00:00:00
Exit pos 10969.441754639496 5.077781314703363 0 0
2015-08-17 00:00:00
Selling Ratio 10969.441754639496 5.189012650630791 -7.

24
192
('20150101', '20160101')
test10000
Return for2015010120160101is10000
('20110101', '20160101')
24
195
('20160101', '20170101')
2016-12-08 00:00:00
Buying Ratio 10000 0.36323780072134226 53.47301225627108 -19.42341936991329
test10000
Return for2016010120170101is10000
('20120101', '20170101')
24
195
('20170101', '20180101')
test10000
Return for2017010120180101is10000
('20130101', '20180101')
4
187
('20180101', '20190101')
2018-11-26 00:00:00
Buying Ratio 10000 0.3463999189471507 50.52216629227552 -17.50087430867871
test10000
Return for2018010120190101is10000
Avg amount over 5 years 10000.0
('banks\\KTKBANK.BO.csv', 'banks\\YESBANK.BO.csv')
('20090101', '20140101')
3
190
('20140101', '20150101')
2014-12-02 00:00:00
Buying Ratio 10000 0.8668283337563434 48.41245959326679 -41.96529168227776
test10000
Return for2014010120150101is10000
('20100101', '20150101')
3
127
('20150101', '20160101')
2015-08-24 00:00:00
Selling Ratio 10000 0.7729246915262344 -58.77903782923652 45.43176968237151
2

3
40
('20170101', '20180101')
2017-04-21 00:00:00
Selling Ratio 10000 0.48565704305622487 -36.27226744523589 17.61588215239783
2017-07-03 00:00:00
Exit pos 9708.282908185247 0.5137652668824934 0 0
2017-07-04 00:00:00
Exit pos 9708.282908185247 0.5141961135685149 0 0
2017-07-05 00:00:00
Exit pos 9708.282908185247 0.5166703569337405 0 0
2017-07-06 00:00:00
Exit pos 9708.282908185247 0.5217456869853785 0 0
2017-07-07 00:00:00
Exit pos 9708.282908185247 0.5255911372088151 0 0
2017-07-10 00:00:00
Exit pos 9708.282908185247 0.5282354830415482 0 0
2017-07-11 00:00:00
Exit pos 9708.282908185247 0.5182062389170019 0 0
2017-07-12 00:00:00
Exit pos 9708.282908185247 0.5218991221143058 0 0
2017-07-13 00:00:00
Exit pos 9708.282908185247 0.4987694118431035 0 0
2017-07-14 00:00:00
Exit pos 9708.282908185247 0.5055525231742763 0 0
2017-07-17 00:00:00
Exit pos 9708.282908185247 0.4890881387302873 0 0
2017-07-18 00:00:00
Exit pos 9708.282908185247 0.48638883794471127 0 0
2017-07-24 00:00:00
Buying Ratio

3
53
('20160101', '20170101')
2016-05-12 00:00:00
Buying Ratio 10000 1.1033146502335374 27.13110811725965 -29.934149062842614
2016-09-09 00:00:00
Exit pos 10067.416273480208 1.1138937388861283 0 0
2016-09-12 00:00:00
Exit pos 10067.416273480208 1.1298162445824997 0 0
2016-09-15 00:00:00
Selling Ratio 10067.416273480208 1.128278140205695 -19.85631688563557 22.40344828705984
2016-10-19 00:00:00
Exit pos 10375.694342573996 1.0629261491854076 0 0
2016-10-20 00:00:00
Exit pos 10375.694342573996 1.070366102847762 0 0
2016-10-21 00:00:00
Exit pos 10375.694342573996 1.0573189035117239 0 0
2016-10-24 00:00:00
Exit pos 10375.694342573996 1.082348376696854 0 0
2016-10-25 00:00:00
Exit pos 10375.694342573996 1.0767061400380171 0 0
2016-10-26 00:00:00
Exit pos 10375.694342573996 1.0832127774541682 0 0
2016-10-27 00:00:00
Exit pos 10375.694342573996 1.095864734070634 0 0
2016-11-07 00:00:00
Selling Ratio 10375.694342573996 1.129254132754619 -20.71539212827766 23.392942172490052
test10375.69434257399

In [52]:
avg = []
training = []
testing = []
arr = []
arr_ct = []

for item in pairs:
    print(item)
    name1 = item[0]
    name2 = item[1]
    data1 = (pd.read_csv(name1,parse_dates = ['Date'],index_col = 'Date')).interpolate(axis=0)
    data2 = (pd.read_csv(name2,parse_dates = ['Date'],index_col = 'Date')).interpolate(axis=0)
    money = 0
    row1 =  []
    row2 =  []
    for i in range(0,5):
        train_p = periods[i]
        value1 = cointegration_test(data1,data2,train_p[0],train_p[1])[0][1]
        value2 = cointegration_test_ct(data1,data2,train_p[0],train_p[1])[0][1]
        row1.append(value1)
        row2.append(value2)
        print(train_p)
        print(value1)
        print(value2)
    arr.append(row1)
    arr_ct.append(row2)


('banks\\ALBK.BO.csv', 'banks\\ANDHRABANK.BO.csv')
('20090101', '20140101')
0.05947777561828621
0.11498131996741723
('20100101', '20150101')
0.0008869060376010418
0.006211388240952663
('20110101', '20160101')
0.032773629172828646
0.023705570500372367
('20120101', '20170101')
0.08735769997283571
0.1094301144936412
('20130101', '20180101')
0.11522972470186443
0.15524937570206326
('banks\\ALBK.BO.csv', 'banks\\AXISBANK.BO.csv')
('20090101', '20140101')
0.8349349463936566
0.7584642413265855
('20100101', '20150101')
0.9972678232490944
0.9070082528678096
('20110101', '20160101')
0.8732619928336595
0.3940220512038081
('20120101', '20170101')
0.8302620187764049
0.21120704346028146
('20130101', '20180101')
0.13546968703992263
0.19868218759525413
('banks\\ALBK.BO.csv', 'banks\\BANKBARODA.BO.csv')
('20090101', '20140101')
0.8735931526952805
0.3620373182604587
('20100101', '20150101')
0.9875988598121236
0.5129735513743777
('20110101', '20160101')
0.6876324707725123
0.027351105384595096
('20120101'

('20090101', '20140101')
0.9798800211565831
0.6947464735962408
('20100101', '20150101')
0.9478004933154617
0.07115127232267618
('20110101', '20160101')
0.8521277819894346
0.013974286303827466
('20120101', '20170101')
0.9049750406980033
0.016348150685798687
('20130101', '20180101')
0.6416759101891638
0.15199049754262028
('banks\\ANDHRABANK.BO.csv', 'banks\\ICICIBANK.BO.csv')
('20090101', '20140101')
0.9736450708020498
0.8409443801152824
('20100101', '20150101')
0.993680066856142
0.5574567466100376
('20110101', '20160101')
0.8277008118211
0.31823396456957764
('20120101', '20170101')
0.7376118879255615
0.6523894895597635
('20130101', '20180101')
0.037392950329191224
0.00438521087310838
('banks\\ANDHRABANK.BO.csv', 'banks\\INDUSINDBK.BO.csv')
('20090101', '20140101')
0.9593130485078158
0.4083493422938928
('20100101', '20150101')
0.9923398536375451
0.3371852901489858
('20110101', '20160101')
0.9583637052062717
0.1580063154779744
('20120101', '20170101')
0.9629788792671439
0.0159220351476498

('20120101', '20170101')
0.5856321654552301
0.02831055257949393
('20130101', '20180101')
0.30659771803965175
0.10772849871220586
('banks\\BANKBARODA.BO.csv', 'banks\\CUB.BO.csv')
('20090101', '20140101')
0.27492629978987954
0.26393284883147017
('20100101', '20150101')
0.04769812185456813
0.02210969678529381
('20110101', '20160101')
0.34930853541028317
0.201508230050218
('20120101', '20170101')
0.7025444876297364
0.4452815575981377
('20130101', '20180101')
0.9661165541129503
0.8156704435417899
('banks\\BANKBARODA.BO.csv', 'banks\\DCBBANK.BO.csv')
('20090101', '20140101')
0.079741473122386
0.028705424594177204
('20100101', '20150101')
0.026382756600961097
0.14939072605159626
('20110101', '20160101')
0.18470416810763185
0.29923902189039503
('20120101', '20170101')
0.14258605759110649
0.09329915648052883
('20130101', '20180101')
0.8326020427762715
0.574004238264529
('banks\\BANKBARODA.BO.csv', 'banks\\HDFCBANK.BO.csv')
('20090101', '20140101')
0.479507016166531
0.4724763144492473
('2010010

0.024863526397847516
('20130101', '20180101')
0.6916203804353063
0.02688105331949011
('banks\\CANBK.BO.csv', 'banks\\DCBBANK.BO.csv')
('20090101', '20140101')
0.5566065414340247
0.7177277612500879
('20100101', '20150101')
0.9970674087525326
0.9830535404793115
('20110101', '20160101')
0.7216377121463156
0.4540908101003385
('20120101', '20170101')
0.7024973854489872
0.22861908576813983
('20130101', '20180101')
0.46896132807863816
0.11994898516164115
('banks\\CANBK.BO.csv', 'banks\\HDFCBANK.BO.csv')
('20090101', '20140101')
0.9458168858624737
0.7716767149625637
('20100101', '20150101')
0.8542882702841084
0.15473205583788435
('20110101', '20160101')
0.8003974607195208
0.09960727897253935
('20120101', '20170101')
0.8378823790607224
0.3021845520718991
('20130101', '20180101')
0.46106983253737577
0.20715902443156203
('banks\\CANBK.BO.csv', 'banks\\ICICIBANK.BO.csv')
('20090101', '20140101')
0.9745182023874143
0.9384773398518516
('20100101', '20150101')
0.9934613599854982
0.5890997186349316
('

('20090101', '20140101')
0.6231378968898668
0.2057424888443885
('20100101', '20150101')
0.7322526793617329
0.3937412811249569
('20110101', '20160101')
0.33807130073005054
0.24032634205443176
('20120101', '20170101')
0.7343059340403245
0.08623269016565133
('20130101', '20180101')
0.17750224073664117
9.91464692941068e-05
('banks\\CUB.BO.csv', 'banks\\SBIN.BO.csv')
('20090101', '20140101')
0.11946081083649024
0.006786689246644752
('20100101', '20150101')
0.012934444268616759
0.025200937987546707
('20110101', '20160101')
0.057385070763098255
0.047144260308062465
('20120101', '20170101')
0.8378281555754465
0.6743895844862746
('20130101', '20180101')
0.8774860511388807
0.5513320356470601
('banks\\CUB.BO.csv', 'banks\\YESBANK.BO.csv')
('20090101', '20140101')
0.0010438723098221746
0.0069443148063897415
('20100101', '20150101')
4.091242681350232e-05
0.0002772226660136851
('20110101', '20160101')
0.015352908679081033
0.003175619012826025
('20120101', '20170101')
0.24900463295617453
0.1433185022

('20090101', '20140101')
0.7964440170211171
0.29974148491936636
('20100101', '20150101')
0.912657932066391
0.6517478898401553
('20110101', '20160101')
0.790132393459629
0.6770376428874396
('20120101', '20170101')
0.8306526563407938
0.30569136209934866
('20130101', '20180101')
0.1590002388817695
0.031843267417789776
('banks\\INDUSINDBK.BO.csv', 'banks\\SBIN.BO.csv')
('20090101', '20140101')
0.5921926521320271
0.026622214276060496
('20100101', '20150101')
0.2423683858527954
0.20077350130525934
('20110101', '20160101')
0.9338281219602512
0.8957239419122638
('20120101', '20170101')
0.7551800933837416
0.6372444065708384
('20130101', '20180101')
0.7538531545533047
0.5462575454751006
('banks\\INDUSINDBK.BO.csv', 'banks\\YESBANK.BO.csv')
('20090101', '20140101')
0.22917038252358934
0.026309014486035325
('20100101', '20150101')
0.0496144629877634
0.2059726895634057
('20110101', '20160101')
0.19760626745060356
0.43405276056295355
('20120101', '20170101')
0.36637403167650984
0.5398697053351911
('

In [56]:
__2015 = []
__2016 = []
__2017 = []
__2018 = []
__2019 = []
for i in arr_ct:
    __2015.append(i[0])
    __2016.append(i[1])
    __2017.append(i[2])
    __2018.append(i[3])
    __2019.append(i[4])
df = pd.DataFrame(list(zip(pairs,__2015,__2016,__2017,__2018,__2019)), 
               columns =['Pairs' , '2015', '2016', '2017', '2018', '2019']) 
df.to_csv('testingdatacoint_ct.csv')

In [45]:
df = pd.DataFrame(list(zip(pairs,training,avg)), 
               columns =['Pairs','Training', 'avg']) 
df.to_csv('latest2.csv')

In [34]:
_2015 = []
_2016 = []
_2017 = []
_2018 = []
_2019 = []
for i in training:
    _2015.append(i[0])
    _2016.append(i[1])
    _2017.append(i[2])
    _2018.append(i[3])
    _2019.append(i[4])
        
        

In [35]:
df = pd.DataFrame(list(zip(pairs,training,avg,_2015,_2016,_2017,_2018,_2019)), 
               columns =['Pairs','Training', 'avg' , '2015', '2016', '2017', '2018', '2019']) 
df.to_csv('testingdata.csv')

,Symbol,Adjclose,Close,High,Low,Open,Volume
Timestamp,,,,,,,
2010-01-04,JSWENERGY.BO,83.8252,100.75,106.4,99.90,102.0,35110000.0
2010-01-05,JSWENERGY.BO,93.8094,112.75,117.4,100.00,101.5,32356100.0
2010-01-06,JSWENERGY.BO,93.9342,112.90,118.0,111.25,113.8,10518100.0
